# Introduction

This notebook demonstrates the use of SQL to analyze data from the TMDB (The Movie Database) SQLite database. The dataset contains comprehensive information about movies, including details about genres, languages, production companies, and cast. Through various SQL queries, we explore different aspects of the data to answer specific questions and gain insights.



## Project Setup
To begin, we'll need to set up our environment and connect to the TMDB SQLite database. This involves installing necessary packages and configuring SQL magic commands for Jupyter notebooks.

In [ ]:
!pip install ipython-sql
!pip install SQLAlchemy<2.0

## Database Schema
The TMDB database consists of several interconnected tables. Here's an overview of the database structure:![SQL Architectures](https://raw.githubusercontent.com/Explore-AI/Public-Data/master/image/TMDB_ERD.JPG)

Once the packages are installed, load the SQL extension to enable SQL queries within Jupyter:

In [ ]:
%load_ext sql

### Connecting to the Database
To connect to the TMDB database file saved in the same location as this notebook, use the following SQL connection string:

In [ ]:
%%sql 
sqlite:///TMDB.db

## Data Exploration and Analysis

Let's dive into some interesting queries to extract valuable insights from our movie database.


### Examining the Movies Table Structure

We'll start by investigating the primary key of the 'movies' table to understand its structure.
Based on the entity-relationship diagram (ERD) provided, the primary key for the "Movies" table is movie_id. This is evident from the diagram where "movie_id" is listed at the top of the Movies table with a key icon next to it, indicating it is the primary key for that table.

### Analyzing Table Relationships

Next, we'll explore the relationships between tables by examining the foreign keys in the 'languagemap' table.

Based on the ERD, the "LanguageMap" table has 2 foreign keys:

movie_id which links to the Movies table
iso_639_1 which links to the Languages table
These foreign keys establish relationships between the LanguageMap table and other tables, ensuring data integrity across the database.


### Creating Views for Unreleased Movies

To facilitate analysis of movies in production or pre-release stages, we'll create a view of all unreleased movies.


In [ ]:
%%sql
CREATE VIEW Not_Released AS 
SELECT * 
FROM movies 
WHERE release_status <> 'Released';

### Exploring Movie Titles and Release Information

We'll query our newly created view to examine titles, release dates, and statuses of unreleased movies.


In [ ]:
%%sql
SELECT title, release_date, release_status 
FROM Not_Released;

### Analyzing Original vs. Released Titles

Let's investigate how many movies have changed their titles from the original.


In [ ]:
%%sql 
SELECT COUNT(*)
FROM movies
WHERE title != original_title;

### Finding the Most Popular Recent Big-Budget Movie

We'll identify the most popular movie released after 2000 with a budget exceeding $100 million.


In [ ]:
%%sql
SELECT title, release_date, budget, popularity
FROM movies
WHERE release_date > '2000-01-01' 
  AND budget > 100000000
ORDER BY popularity DESC
LIMIT 1;

### Identifying High-Grossing Movies by Genre
We analyze which genres have produced the highest-grossing movies. This helps in understanding the financial success associated with different genres.

In [ ]:
%%sql
SELECT COUNT(*)
FROM movies
WHERE original_language <> 'en';

### Number of Movies Produced by Pixar Animation Studios
We want to determine how many movies in the database were produced by Pixar Animation Studios.

In [ ]:
%%sql
SELECT COUNT(*)
FROM Movies AS m
JOIN ProductionCompanyMap AS pcm ON m.movie_id = pcm.movie_id
JOIN ProductionCompanies AS pc ON pcm.production_company_id = pc.production_company_id
WHERE pc.production_company_name = 'Pixar Animation Studios';

### Number of Movies That Are Both a Romance and a Comedy


In [ ]:
%%sql
SELECT COUNT(*)
FROM Movies AS m
JOIN GenreMap AS gm1 ON m.movie_id = gm1.movie_id
JOIN GenreMap AS gm2 ON m.movie_id = gm2.movie_id
JOIN Genres AS g1 ON gm1.genre_id = g1.genre_id
JOIN Genres AS g2 ON gm2.genre_id = g2.genre_id
WHERE g1.genre_name = 'Romance'
  AND g2.genre_name = 'Comedy';

### Most Popular Action Movie with Some German Language


In [ ]:
%%sql
SELECT m.title, m.popularity
FROM Movies m
JOIN LanguageMap lm ON m.movie_id = lm.movie_id
JOIN Languages l ON lm.iso_639_1 = l.iso_639_1
JOIN GenreMap gm ON m.movie_id = gm.movie_id
JOIN Genres g ON gm.genre_id = g.genre_id
WHERE l.language_name = 'Deutsch'
  AND g.genre_name = 'Action'
ORDER BY m.popularity DESC
LIMIT 1;


### Movies Featuring Tom Cruise as Ethan Hunt

In [ ]:
%%sql 
SELECT COUNT(DISTINCT c.movie_id) AS ethan_hunt_movies
FROM Actors a
JOIN Casts c ON a.actor_id = c.actor_id
WHERE a.actor_name = 'Tom Cruise'
AND c.characters LIKE '%Ethan Hunt%';

### Cate Blanchett's Oscar Nominations
To determine the number of Oscar nominations received by Cate Blanchett, the following query was executed:

In [ ]:
%%sql 
SELECT COUNT(*) AS nomination_count
FROM Oscars
WHERE name = 'Cate Blanchett';

This query counts the number of nominations for Cate Blanchett by querying the Oscars table where the name column matches 'Cate Blanchett'.

### Movies with South African Languages

In [ ]:
%%sql
SELECT COUNT(DISTINCT lm.movie_id) AS south_african_language_movies
FROM LanguageMap AS lm
JOIN Languages AS l ON lm.iso_639_1 = l.iso_639_1
WHERE l.language_name IN ('Afrikaans', 'isiZulu');  -- Adjust 'Zulu' to 'isiZulu' based on findings


This query counts the number of distinct movies that have either Afrikaans or isiZulu as one of their languages by joining the LanguageMap and Languages tables and filtering by the specified language names.

### Most Popular Movie with South African Languages

In [ ]:
%%sql
SELECT title, popularity
FROM Movies
WHERE title IN ('Blood Diamond', 'Tsotsi', 'Gangster''s Paradise: Jerusalema', 'District 9')
ORDER BY popularity DESC
LIMIT 1;

### Updating Language Name
To update the name of the language with the ‘zh’ ISO code to ‘Chinese’, use the following SQL code:

In [ ]:
%%sql 
UPDATE languages SET language_name = 'Chinese' WHERE iso_639_1 = 'zh';

### Adding a New Genre

In [ ]:
%%sql 
INSERT INTO genres (genre_id, genre_name) VALUES (10, 'Sport');

This query inserts a new record into the genres table with the genre_id of 10 and genre_name of 'Sport'.

### Deleting Genre Link for "The Flintstones"


In [ ]:
%%sql 
DELETE FROM genremap WHERE genre_id = 35 AND movie_id = 888;

This query removes the link between the movie "The Flintstones" (with movie_id 888) and the Comedy genre (with genre_id 35) from the genremap table.

### Most Recent Movies

In [ ]:
%%sql 
SELECT * FROM movies ORDER BY release_date DESC LIMIT 10;

This query retrieves the latest 10 movies by ordering the results in descending order of the release_date and limiting the output to 10 records.

 ### Adding English Name Column to Languages Table

In [ ]:
%%sql 
ALTER TABLE languages ADD language_english_name varchar(50);

This query modifies the languages table to include an additional column called language_english_name for storing English names of the languages.

## Conclusion

This project showcases various SQL techniques for querying and manipulating a complex movie database. From basic select statements to creating views and modifying data, we've explored a wide range of SQL functionalities. These skills are crucial for data analysts and database administrators working with large datasets in the entertainment industry or similar fields.

## Future Work

Potential extensions of this project could include:
- Implementing more complex joins to analyze relationships between actors, directors, and movie success
- Creating stored procedures for common query patterns
- Developing a simple front-end interface for non-technical users to query the database
- Integrating with current TMDB API to keep the database updated
